In [1]:
# Modules
import pandas as pd
import requests
import matplotlib as plt
import json
import time
from datetime import date
from dateutil.relativedelta import relativedelta

In [2]:
# API Key
from keys import NY_Times_API_KEY

In [3]:
# Request Settings
from settings import startDate, endDate

In [4]:
# Base URLs

NY_Times_ARCHIVE_BASE_URL = 'https://api.nytimes.com/svc/archive/v1/'#YYYY/MM.json?api-key=
NY_Times_ARTICLE_SEARCH_BASE_URL = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q='

Hypothesis: Articles released by the NY Times influence the performance of stock symbols.
Null Hypothesis: Articles released by the NY Times have no correlation with the performance of stock symbols.

In [5]:
def request_articles(date):
    base = NY_Times_ARCHIVE_BASE_URL
    url = f'{base}{date.year}/{date.month}.json?api-key={NY_Times_API_KEY}'
    try:
        req = requests.get(url).json()
    except Exception as e:
        return (e,'')
    time.sleep(5)
    return req

In [6]:
def has_headline(article):
    article_has_headline = type(article[ 'headline']) == dict and 'main' in article['headline'].keys()
    return article_has_headline

In [7]:
def first_of_months_in_date_range(startDate=date.today(),endDate=date.today()):
    months_in_range = [x.split(' ') for x in pd.date_range(startDate, endDate, freq='MS').strftime("%Y %m %d").tolist()]
    return months_in_range

In [8]:
article_date_range = first_of_months_in_date_range(startDate=startDate,endDate=endDate)


In [9]:
articles = { # Column headers for dataframe
    'id':[],
    'date':[],
    'url':[],
    'headline':[],
    'abstract':[],
    'leadParagraph':[],
    'keywords':[],
    'author':[]
}

In [10]:
y = 1
b = tuple()
for d in article_date_range: # requests articles for every month in the range of startDate to endDate variables defined above. Filters by has_headline
    year, month, day  = int(d[0]), int(d[1]), int(d[2])
    a = request_articles(date(year,month,day)) # Hands back either the request json 
    if type(a) == type(b):
        print(f'REQUEST ERROR: {a[0]}')
    else:
        for article in a:
            for i in range(1,len(article)):
                try:
                    articles['index']         = y
                    articles['id']            = article[i]['_id']
                    articles['date']          = article[i]['pub_date']
                    articles['url']           = article[i]['web_url']
                    articles['headline']      = article[i]['headline']
                    articles['abstract']      = article[i]['abstract']
                    articles['leadParagraph'] = article[i]['lead_paragraph']
                    articles['keywords']      = article[i]['keywords']
                    articles['author']        = article[i]['byline']
                except Exception as e:
                    print(f'Exception when assigning data to articles dict: {e}')
            y += 1

Exception when assigning data to articles dict: string indices must be integers
Exception when assigning data to articles dict: string indices must be integers
Exception when assigning data to articles dict: string indices must be integers
Exception when assigning data to articles dict: string indices must be integers
Exception when assigning data to articles dict: string indices must be integers
Exception when assigning data to articles dict: string indices must be integers
Exception when assigning data to articles dict: string indices must be integers
Exception when assigning data to articles dict: string indices must be integers
Exception when assigning data to articles dict: string indices must be integers
Exception when assigning data to articles dict: string indices must be integers
Exception when assigning data to articles dict: string indices must be integers
Exception when assigning data to articles dict: string indices must be integers
Exception when assigning data to article

In [11]:
allArticlesDataFrame = pd.DataFrame(data=articles,index=articles['date'])
allArticlesDataFrame.head()

,id,date,url,headline,abstract,leadParagraph,keywords,author,index


In [12]:
_ = allArticlesDataFrame.to_csv('articles.csv')